In [2]:
PROJECT_ID = 'dwh-siloam'
REGION = 'asia-southeast1'
print(f"Project ID: {PROJECT_ID}\nRegion: {REGION}")

# Initialize Vertex AI
from pathlib import Path
import vertexai
from google.cloud import aiplatform

print(f"Checking Credentials...")
if not any((Path.cwd()/"service_account").glob('*.json')):
    print("Service account folder is empty. Fallback using default gcloud account")
    aiplatform.init(project=PROJECT_ID, location=REGION)
    vertexai.init(project=PROJECT_ID, location=REGION)
else:
    print('Using service account credentials from service_account folder')
    from google.oauth2 import service_account
    sa_file = list((Path.cwd()/"service_account").glob('*.json'))[0]
    print(f"Using service account file: {sa_file}")
    credentials = service_account.Credentials.from_service_account_file(sa_file)
    aiplatform.init(project=PROJECT_ID, location=REGION, credentials=credentials)
    vertexai.init(project=PROJECT_ID, location=REGION, credentials=credentials)

# Import libraries
from langchain_google_vertexai import VertexAI, ChatVertexAI, create_structured_runnable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional
import requests

from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder
from settings import CopilotSettings
import time

Project ID: dwh-siloam
Region: asia-southeast1
Checking Credentials...
Using service account credentials from service_account folder
Using service account file: c:\Others\chatbot-llm\service_account\dwh-siloam-99402e61edd2.json


In [3]:
config = CopilotSettings()

## Initialize model & prompt template

In [4]:
llm = ChatVertexAI(
    model_name="chat-bison", 
    temperature=0, 
    max_output_tokens=8192
)

In [5]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate


In [8]:
prompt1 = hub.pull("jet-taekyo-lee/time-aware-react-multi-input-json")
prompt1.input_variables

['agent_scratchpad', 'input', 'tool_names', 'tools']

In [9]:
prompt1.pretty_print()

================================ System Message ================================

Respond to the human as helpfully and accurately as possible in the same language as the human. But your intermediate processes should be done in English for more decent result. You have access to the following tools:
{tools}
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or {tool_names}
Provide only ONE action per $JSON_BLOB, as shown:
```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```
Follow this format:
Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Final response to human"
}
Please keep the following checkpoints in mind: 
Checkpoint 1) No matter what, your FIRST Th

In [10]:
prompt2 = hub.pull("hwchase17/openai-tools-agent")
prompt2.input_variables

['agent_scratchpad', 'input']

In [11]:
prompt2.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


## Create tools

In [12]:
from langchain_core.tools import tool

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [multiply, add, exponentiate]

In [15]:
tools[0].description

'multiply(first_int: int, second_int: int) -> int - Multiply two integers together.'

## Using Agents

In [13]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [16]:
# llm_w_tools = llm.bind_tools(tools)
# Construct the tool calling agent
agent = create_tool_calling_agent(
    llm=llm, 
    tools=tools, 
    prompt=prompt2, 
)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True
)

In [17]:
agent_executor.invoke(
    {
        "input": "Alice has 5 apples, and she ate two of them. How many apples are there?"
    }
)



> Entering new AgentExecutor chain...


TypeError: _ChatModelBase.start_chat() got an unexpected keyword argument 'functions'